In [1]:
import ee
import geemap
from geeRiverCL import *
import numpy as np
geemap.ee_initialize()

In [2]:
# roi
ltb = ee.FeatureCollection("users/luoyee1997/SF").geometry()
# Planet Scope
imgPS = load_image('users/luoyee1997/geeRiver/SF/PlanetSR', ['b1', 'b2', 'b3', 'b4'], ['Blue', 'Green', 'Red', 'NIR']).clip(ltb).multiply(0.0001)
crsPS = imgPS.projection().crs().getInfo()
sclPS = 3

In [3]:
Map = geemap.Map()

In [4]:
vis = {
    'bands': ['Red', 'Green', 'Blue'],
    'min': 0,
    'max': 0.7231,
    'gamma': 2
}
Map.centerObject(imgPS, 12)
Map.addLayer(imgPS, vis, 'Landsat')

In [5]:
# classification
classPS = classification1(imgPS, 3, method='RF')

In [5]:
# classification
classPS = classification1(imgPS, 3, method='minDis')

In [5]:
# classification
classPS = classification1(imgPS, 3, method='RF')

In [6]:
Map.addLayer(classPS, {'min':1, 'max':3, 'palette': ['#0080FF', 'red', 'green']}, 'classPS')

In [9]:
ndvi = imgPS.normalizedDifference(['NIR', 'Red']).rename('NDVI')
ndwi = imgPS.normalizedDifference(['Green', 'NIR']).rename('NDWI')
evi = Evi(imgPS).rename('EVI')

In [19]:
Map.addLayer(ndvi.lt(0.2).selfMask(), {}, 'ndvi')

In [17]:
Map.addLayer(evi.lt(0).selfMask(), {}, 'evi')

In [14]:
Map.addLayer(ndvi.lt(0.3).And(ndvi.gt(0.25)).And(evi.lt(0.3)).selfMask(), {}, 'sand')

In [8]:
Map

Map(center=[33.19408632446534, -90.67425695260197], controls=(WidgetControl(options=['position', 'transparent_…

In [7]:
# obtain river masks
waterPS = classPS.eq(1)
waterPS2 = close(waterPS, 3, radius=3)
riverPS = noise_removal(waterPS2, 3, 1000, 4)

In [8]:
# fill holes of river masks
notRiver = riverPS.unmask().Not()
riverPS1 = noise_removal(notRiver, 3, 50, 4)
riverPS2 = riverPS1.unmask().Not()

In [9]:
Map.addLayer(riverPS2.selfMask().reproject(crs = crsPS, scale = sclPS), {}, 'river')

In [10]:
# calculate centerline
cl = CalculateCenterline(riverPS2.unmask(), sclPS, thre=0.9)

In [11]:
# trim unwanted branches
trimed = trim_centerline(cl, sclPS, 5).selfMask()

In [12]:
Map = geemap.Map()

In [13]:
Map.addLayer(trimed.reproject(crs = crsPS, scale = sclPS), {'palette': ['blue']}, 'trimed')

In [16]:
Map.addLayer(cl.reproject(crs = crsPS, scale = sclPS), {'palette': ['blue']}, 'cl')

In [14]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…